In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.rcParams["figure.figsize"] = (15, 5)
from pmdarima import auto_arima, model_selection, ARIMA
from pmdarima.arima.utils import ndiffs
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from datetime import datetime, timedelta, date
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import joblib

In [151]:
countries = 'Denmark|Spain'
datecol = 'date'
exog=pd.read_csv("data/exogenous.csv", parse_dates=[datecol], index_col=[datecol])
endog=pd.read_csv("data/endogenous.csv", parse_dates=[datecol], index_col=[datecol])
cases="cases"

for country in countries.split('|'):
    filename = './models/' + country + 'SARIMAXmodel.pkl'
    model = joblib.load(filename)
    if model.fittedvalues.index[-1]== exog.index[-1]:
        print("up to date")
    else:
        
        exogmodel=exog.loc[:, exog.columns.str.contains(country)]
        endogmodel=endog.loc[:, endog.columns.str.contains("new_{}_{}".format(cases,country))]
        exogmodel=exogmodel.iloc[-1:,:]
        sc_in = MinMaxScaler(feature_range=(0, 1))
        scaled_input = sc_in.fit_transform(exogmodel)
        scaled_input = pd.DataFrame(scaled_input, index=exogmodel.index, columns=exogmodel.columns)
        X_update = scaled_input
        X_update=X_update.iloc[-1:,:]
        
        sc_out = MinMaxScaler(feature_range=(0, 1))
        scaled_output = sc_out.fit_transform(endogmodel)
        scaled_output  = pd.Series(scaled_output.flatten(), index=endogmodel.index, name=endogmodel.columns[0])
        y = scaled_output.resample('1D').sum()
        y=y.iloc[-1:]
        modelupdated=model.append(y, exog=X_update, refit=True)
        joblib.dump(modelupdated, filename)

up to date


In [135]:
filename = './models/' + "Denmark" + 'SARIMAXmodel.pkl'
model = joblib.load(filename)

In [155]:
filename = './models/' + "Spain" + 'SARIMAXmodel.pkl'
model = joblib.load(filename)

In [156]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:        new_cases_Spain   No. Observations:                  259
Model:               SARIMAX(1, 1, 3)   Log Likelihood                 720.413
Date:                Wed, 16 Sep 2020   AIC                          -1402.826
Time:                        01:00:46   BIC                          -1335.320
Sample:                    01-01-2020   HQIC                         -1375.681
                         - 09-15-2020                                         
Covariance Type:                  opg                                         
==============================================================================================================
                                                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
positive_rate_Spain                            0.1598      0.071      2.248      0.025       0.020       0.299
C1_School closing_Spain                        0.0061      0.164      0.037      0.970      -0.315       0.327
C2_Workplace closing_Spain                     0.0133      0.147      0.090      0.928      -0.275       0.302
C3_Cancel public events_Spain                  0.2515      0.283      0.889      0.374      -0.303       0.806
C4_Restrictions on gatherings_Spain           -0.1034      0.108     -0.961      0.337      -0.314       0.108
C5_Close public transport_Spain               -0.0560      0.117     -0.479      0.632      -0.285       0.173
C6_Stay at home requirements_Spain             0.0797      0.064      1.251      0.211      -0.045       0.205
C7_Restrictions on internal movement_Spain     0.0382      0.118      0.325      0.745      -0.193       0.269
C8_International travel controls_Spain        -0.3240      0.360     -0.899      0.369      -1.030       0.382
H1_Public information campaigns_Spain          0.0179      1.285      0.014      0.989      -2.502       2.537
H2_Testing policy_Spain                       -0.1100      0.065     -1.697      0.090      -0.237       0.017
H3_Contact tracing_Spain                      -0.0443      1.233     -0.036      0.971      -2.462       2.373
mask_Spain                                     0.7537      0.508      1.484      0.138      -0.242       1.749
flightsArr_Spain                              -0.1601      0.276     -0.580      0.562      -0.701       0.381
ar.L1                                          0.8860      0.039     22.691      0.000       0.809       0.963
ma.L1                                         -0.6628      0.067     -9.963      0.000      -0.793      -0.532
ma.L2                                          0.0245      0.066      0.374      0.709      -0.104       0.153
ma.L3                                          0.1243      0.053      2.332      0.020       0.020       0.229
sigma2                                         0.0002   1.15e-05     18.609      0.000       0.000       0.000
===================================================================================
Ljung-Box (Q):                       63.80   Jarque-Bera (JB):              1140.24
Prob(Q):                              0.01   Prob(JB):                         0.00
Heteroskedasticity (H):              10.36   Skew:                             1.60
Prob(H) (two-sided):                  0.00   Kurtosis:                        12.79
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""